In [1]:
import antares

Loading ANTARES from /data0/sw/antares-kernel/lib/python3.9/site-packages/antares/__init__.py

        _    _   _ _____  _    ____  _____ ____
       / \  | \ | |_   _|/ \  |  _ \| ____/ ___|
      / _ \ |  \| | | | / _ \ | |_| |  _| \___ \\
     / ___ \| |\  | | |/ ___ \|  _ /| |___ ___| |
    /_/   \_\_| \_| |_/_/   \_\_| \_\_____|____/   v2.11.0
    


In [2]:
# Imports
import antares.devkit as dk
dk.init()
# You should see a happy message that says that "ANTARES DevKit is ready!"

2025-05-13 07:30:25,945 - WARNING MainThread settings.py:setup_prometheus:124 - Prometheus failed to start with [Errno 98] Address already in use
2025-05-13 07:30:26,068 - WARNING MainThread settings.py:setup_prometheus:124 - Prometheus failed to start with [Errno 98] Address already in use


Jaeger tracer already initialized, skipping


Testing loading a random Locus with `dk.get_locus()`...

ANTARES v2.11.0 DevKit is ready!
Website: https://antares.noirlab.edu
Documentation: https://nsf-noirlab.gitlab.io/csdc/antares/antares/



In [49]:
# Define a Paczyński microlensing model
def paczynski(t, t0, u0, tE, F_s):
    """
    Paczyński microlensing light curve model
    t0 : peak time
    u0 : impact parameter
    tE : Einstein crossing time
    F_s : source flux
    F_b : blended flux
    """
    u = np.sqrt(u0**2 + ((t - t0) / tE) ** 2)
    A = (u**2 + 2) / (u * np.sqrt(u**2 + 4))
    return F_s * (A - 1) + (1-F_s)

def mag_to_flux(mag, F0=1.0):
    """
    Convert magnitude to flux.
    
    Parameters:
    - mag : magnitude (float or array)
    - F0 : reference flux (zeropoint), default=1.0 for relative flux
    
    Returns:
    - flux : flux corresponding to the magnitude
    """
    flux = F0 * 10**(-0.4 * mag)
    flux = flux/np.min(flux)
    return flux
    
def magerr_to_fluxerr(mag, mag_err, F0=1.0):
    """
    Convert magnitude uncertainty to flux uncertainty.
    
    Parameters:
    - mag : magnitude value or array
    - mag_err : magnitude uncertainty value or array
    - F0 : zeropoint flux (default=1.0 for relative flux)
    
    Returns:
    - flux_err : flux uncertainty
    """
    flux = mag_to_flux(mag, F0)
    flux_err = 0.4 * np.log(10) * flux * mag_err
    return flux_err

# Writing a preliminary microlensing filter that reads in the photometry

In [79]:
from statsmodels.stats.weightstats import DescrStatsW
import numpy as np
from astropy.table import MaskedColumn
import warnings
import astropy
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.stats import skew

class microlensing(dk.Filter):    
    INPUT_LOCUS_PROPERTIES = [
        'ztf_object_id',
    ]
    
    OUTPUT_TAGS = [
        {
            'name': 'microlensing_candidate',
            'description': 'Locus - a transient candidate - exhibits a microlensing-like variability',
        }
    ]


    def make_lc(self, locus):

        with warnings.catch_warnings():
            # The cast of locus.timeseries: astropy.table.Table to a pandas
            # dataframe results in the conversion of some integer-valued
            # columns to floating point represntation. This can result in a
            # number of noisy warning so we will catch & ignore them for the
            # next couple of lines.
            warnings.simplefilter("ignore", astropy.table.TableReplaceWarning)
            df = locus.timeseries.to_pandas()

        data = df[['ant_mjd', 'ztf_fid', 'ztf_magpsf', 'ztf_sigmapsf']]
        
        dn = data.dropna()
        times=dn['ant_mjd'][dn['ztf_fid']==1]
        mags = dn['ztf_magpsf'][dn['ztf_fid']==1]
        mags_err = dn['ztf_sigmapsf'][dn['ztf_fid']==1]
        flxs = mag_to_flux(mags)
        flx_errs = magerr_to_fluxerr(mags, mags_err)
        
        t0_guess = times[np.argmin(mags)]  # Min mag is peak time
        u0_guess = 1/(np.max(flxs))

        initial_fit  = paczynski(times,
                                     t0_guess, 
                                     u0_guess, 
                                     20, 
                                     0.5)

        # plt.gca().invert_yaxis()
        plt.scatter(times, 
                    flxs, color='g', label='g_band')
        plt.plot(times, 
                 initial_fit, color='b', label='initial fit')
        
        plt.xlabel('Time (mjd)')
        plt.ylabel('Flux')
        plt.legend()


    def is_microlensing_candidate(self, times, mags, errors):
        """
        Example of a set of Microlensing detection criteria
        """
        if len(times) < 10:  # Too few data points
            return False

        # Sort data by time
        sorted_idx = np.argsort(times)
        times, mags, errors = times[sorted_idx], mags[sorted_idx], errors[sorted_idx]

        # 1. Check for smoothness (low skewness means symmetric light curve)
        if abs(skew(mags)) > 1:
            return False

        # 2. Check variability (microlensing should have a clear peak)
        if np.ptp(mags) < 0.5:  # Peak-to-peak magnitude difference
            return False

        flxs = mag_to_flux(mags)
        flx_errs = magerr_to_fluxerr(mags, errors)

        # 3. Perform a lightweight template fit (Paczyński model)
        t0_guess = times[np.argmin(mags)]  # Min mag is peak time
        u0_guess = 1/(np.max(flxs))
        initial_guess = [t0_guess, 
                         u0_guess, 
                         20, 
                         0.5]  # Initial params

        # try:
        popt, _ = curve_fit(paczynski, times, flxs, p0=initial_guess, sigma=flx_errs)
        chi2 = np.sum(((flxs - paczynski(times, *popt)) / flx_errs) ** 2) / len(times)

        # 4. Apply a simple chi2 threshold
        if chi2 < 2:  # Well-fit light curves pass
            return True
        # except RuntimeError:
        #     return False  # Fit failed

        return False
    def run(self, locus):
        print('Processing Locus:', locus.locus_id)

        with warnings.catch_warnings():
            warnings.simplefilter("ignore", astropy.table.TableReplaceWarning)
            df = locus.timeseries.to_pandas()

        data = df[['ant_mjd', 'ztf_fid', 'ztf_magpsf', 'ztf_sigmapsf']].dropna()

        
        
        # Split into g-band and i-band
        for band in [1, 2]:  # 1 = g-band, 2 = i-band
            band_data = data[data['ztf_fid'] == band]
            times, mags, errors = band_data['ant_mjd'].values, band_data['ztf_magpsf'].values, band_data['ztf_sigmapsf'].values
            
            if self.is_microlensing_candidate(times, mags, errors):
                print(f'Locus {locus.locus_id} is a microlensing candidate in band {band}')
                locus.tag('microlensing_candidate')
        
        
        return

In [80]:
# Execute the microlensing filter on the locus
report = dk.run_filter(microlensing, locus="ANT2023wuk92lk9fz76")


Processing Locus: ANT2023wuk92lk9fz76
Locus ANT2023wuk92lk9fz76 is a microlensing candidate in band 1
Locus ANT2023wuk92lk9fz76 is a microlensing candidate in band 2


/data0/sw/antares-kernel/lib/python3.9/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [81]:
print(report['new_tags'])

{'microlensing_candidate'}


In [82]:
report = dk.run_many(microlensing, n=10)

# `run_many()` returns a report of what the filter did. Take a look at it:
print(report)

Processing Locus: ANT2019klw2u
Processing Locus: ANT2020b2ppw


/data0/sw/antares-kernel/lib/python3.9/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


Processing Locus: ANT2020vrqwg
Processing Locus: ANT2020ar224


/data0/sw/antares-kernel/lib/python3.9/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


Processing Locus: ANT2020bvtk2
Processing Locus: ANT2020afaa7ci


/data0/sw/antares-kernel/lib/python3.9/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


Processing Locus: ANT2020aesodfa
Processing Locus: ANT2020caxcg
Processing Locus: ANT2020b23ci
Filter crashed:

Optimal parameters not found: Number of calls to function has reached maxfev = 1000.


/data0/sw/antares-kernel/lib/python3.9/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


Processing Locus: ANT2020aryaq
{'n': 10, 'results': [{'locus_id': 'ANT2019klw2u', 'locus_data': FilterContext(locus_id="ANT2019klw2u"), 't': 0.060441900999997245, 'new_locus_properties': {}, 'new_alert_properties': {}, 'new_tags': set(), 'raised_halt': False}, {'locus_id': 'ANT2020b2ppw', 'locus_data': FilterContext(locus_id="ANT2020b2ppw"), 't': 0.0526519780000001, 'new_locus_properties': {}, 'new_alert_properties': {}, 'new_tags': set(), 'raised_halt': False}, {'locus_id': 'ANT2020vrqwg', 'locus_data': FilterContext(locus_id="ANT2020vrqwg"), 't': 0.05779401899999925, 'new_locus_properties': {}, 'new_alert_properties': {}, 'new_tags': set(), 'raised_halt': False}, {'locus_id': 'ANT2020ar224', 'locus_data': FilterContext(locus_id="ANT2020ar224"), 't': 0.05578459800000246, 'new_locus_properties': {}, 'new_alert_properties': {}, 'new_tags': set(), 'raised_halt': False}, {'locus_id': 'ANT2020bvtk2', 'locus_data': FilterContext(locus_id="ANT2020bvtk2"), 't': 0.06708379300000189, 'new_locus